In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm, probplot, skew
from scipy.special import boxcox1p
import warnings
warnings.filterwarnings('ignore')

In [6]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [7]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [8]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


# Data Type

In [9]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [10]:
value_counts = []
for c in train.columns:
    value_counts.append(pd.DataFrame(train[c].value_counts()))

In [11]:
from IPython.core.display import HTML

def multi_table(table_list):
    return HTML(
        f"<table><tr> {''.join(['<td>' + table._repr_html_() + '</td>' for table in table_list])} </tr></table>")

In [12]:
multi_table(value_counts)

,PassengerId
1,1
599,1
588,1
589,1
590,1
...,...
301,1
302,1
303,1
304,1


In [13]:
train[['Sex', 'SibSp', 'Parch', 'Ticket', 'Cabin', 'Embarked']] = train[['Sex', 'SibSp', 'Parch', 'Ticket', 'Cabin', 'Embarked']].astype('category')
test[['Sex', 'SibSp', 'Parch', 'Ticket', 'Cabin', 'Embarked']] = test[['Sex', 'SibSp', 'Parch', 'Ticket', 'Cabin', 'Embarked']].astype('category')

In [14]:
train.dtypes

PassengerId       int64
Survived          int64
Pclass            int64
Name             object
Sex            category
Age             float64
SibSp          category
Parch          category
Ticket         category
Fare            float64
Cabin          category
Embarked       category
dtype: object

# missing value

In [15]:
data = pd.concat((train, test))
miss_cnt = data.isnull().sum().sort_values(ascending=False)
miss_pct = miss_cnt / data.shape[0]
miss_table = pd.DataFrame([miss_cnt, miss_pct], index=['count', 'percent']).T
miss_table['count'] = miss_table['count'].astype('int')
miss_table

,count,percent
Cabin,1014,0.774637
Survived,418,0.319328
Age,263,0.200917
Embarked,2,0.001528
Fare,1,0.000764
PassengerId,0,0.000000
Pclass,0,0.000000
Name,0,0.000000
Sex,0,0.000000
SibSp,0,0.000000


In [16]:
data_cabins = data['Cabin']
data.drop(['Cabin'], axis=1, inplace=True)
data = data.reset_index(drop=True)

data.loc[data['Age'].isnull(), 'Age'] = data.groupby(['Pclass', 'Sex', 'SibSp'])['Age'].transform('median')
data['Embarked'].fillna(data['Embarked'].mode()[0], inplace=True)
data['Fare'].fillna(data.Fare.mean(), inplace=True)

# Engineering

In [17]:
data.drop(['PassengerId', 'Name'], axis=1, inplace=True)

In [18]:
data['FamilySize'] = data.SibSp.astype('int') + data.Parch.astype('int')

In [19]:
conditions_0 = [(data['FamilySize'] == 0),
                (data['FamilySize'] >= 1) & (data['FamilySize'] < 3),
                (data['FamilySize'] == 3),
                (data['FamilySize'] > 3)]

values_0 = ['Alone', 'Small', 'Medium', 'Large']

In [20]:
data['Family_s'] = np.select(conditions_0, values_0)

In [21]:
data.drop(['SibSp', 'Parch', 'Ticket', 'FamilySize'], axis=1, inplace=True)

In [22]:
data = pd.get_dummies(data)

In [23]:
train = data.iloc[:train.shape[0], :]
X_test = data.iloc[train.shape[0]:, :]
X_train, y_train = train.drop(['Survived'], axis=1), train.Survived

# Modeling

In [24]:
from sklearn.model_selection import train_test_split

In [25]:
x_train, x_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [26]:
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import auc, accuracy_score, roc_auc_score
import xgboost

In [27]:
xgb_params = {
    'learning_rate': .1,
    'n_estimators': 1000,
    'max_depth': 5,
    'min_child_weight': 1,
    'gamma': 0,
    'subsample': .8,
    'colsample_bytree': .8,
    'scale_pos_weight': 1,
    'objective': 'binary:logistic'
}

model_xgb = XGBClassifier(**xgb_params)
cvresult = xgboost.cv(model_xgb.get_xgb_params(), 
                      xgboost.DMatrix(x_train, y_train),
                      early_stopping_rounds=50, metrics='auc', nfold=5,
                      verbose_eval=0,
                      num_boost_round=model_xgb.get_params()['n_estimators'])

In [28]:
cvresult[cvresult['test-auc-mean'] == cvresult['test-auc-mean'].max()]

,train-auc-mean,train-auc-std,test-auc-mean,test-auc-std
41,0.948154,0.004312,0.869541,0.029081


In [29]:
model_xgb.set_params(n_estimators=cvresult.shape[0])
model_xgb.fit(x_train, y_train, eval_metric='auc')
y_pred = model_xgb.predict(x_train)
y_test_pred = model_xgb.predict(x_test)
y_predprob = model_xgb.predict_proba(x_train)

print(f'Accuracy : {accuracy_score(y_train, y_pred)}')
print(f'AUC Score (train): {roc_auc_score(y_train, y_pred)}')
print(f'AUC Score (test): {roc_auc_score(y_test, y_test_pred)}')
feature_importance = pd.DataFrame(model_xgb.feature_importances_, index=x_train.columns)[0].sort_values(ascending=False)

Accuracy : 0.8862359550561798
AUC Score (train): 0.8621924162968939
AUC Score (test): 0.7962033462033462


In [ ]:
sns.barplot(x=[1,2],y=[1,2])

<AxesSubplot:>

In [ ]:
plt.subplots(figsize=(20, 5))
sns.barplot(x=feature_importance.index, y=feature_importance.values)
plt.title('feature importance')
plt.show()

learning rate: 0.1 -> depth: 80  
Note that this value might be too hight for you depending on the power of your system. In tah case you can increase the learning rate and re-run the command to get the reduced number of estimators.  

In [ ]:
xgb_params['learning_rate'] = 0.15

model_xgb = XGBClassifier(**xgb_params)
cvresult = xgboost.cv(model_xgb.get_xgb_params(), 
                      xgboost.DMatrix(x_train, y_train),
                      early_stopping_rounds=50, metrics='auc', nfold=5,
                      verbose_eval=0,
                      num_boost_round=model_xgb.get_params()['n_estimators'])

In [ ]:
cvresult[cvresult['test-auc-mean'] == cvresult['test-auc-mean'].max()]

In [ ]:
# xgb_params['n_estimators'] = 53

# xgb_grid_params = {
#     'max_depth': range(3, 10, 2),
#     'min_child_weight': range(1, 6, 2)
# }


# search_xgb = GridSearchCV(model_xgb, xgb_grid_params, cv=5, verbose=1, n_jobs=-1)
# search_xgb.fit(x_train, y_train)

In [ ]:
from sklearn.model_selection import RepeatedStratifiedKFold

In [ ]:
# CV = RepeatedStratifiedKFold(n_splits = 10, n_repeats = 3, random_state = 999)

# XGB_0 = XGBClassifier(use_label_encoder = False, 
#                           objective = 'binary:logistic')

# XGB_param_0 = {'n_estimators': [1000],
#                'learning_rate': [0.01, 0.05, 0.07, 0.1, 0.15, 0.2],
#                'max_depth': [5],
#                'min_child_weight': [1],
#                'gamma': [0],
#                'subsample': [0.8],
#                'colsample_bytree': [0.8]}

# XGB_grid_0 = GridSearchCV(XGB_0, XGB_param_0, verbose=0, 
#                           scoring='neg_log_loss', cv=CV, n_jobs=-1) # n_jobs = 6

# XGB_tuned_0 = XGB_grid_0.fit(x_train, y_train, early_stopping_rounds = 15, verbose=0,
#                               eval_set = [[x_test, y_test]], eval_metric = 'logloss')

In [ ]:
CV = RepeatedStratifiedKFold(n_splits = 10, n_repeats = 3, random_state = 999)

XGB_0 = XGBClassifier(use_label_encoder = False, 
                          objective = 'binary:logistic')

XGB_param_0 = {'n_estimators': [20, 50, 100, 200, 300],
               'learning_rate': [0.01, 0.05, 0.07, 0.1],
               'max_depth': [4],
               'min_child_weight': [4],
               'gamma': [0.2],
               'subsample': [0.9],
               'colsample_bytree': [0.9]}

XGB_grid_0 = GridSearchCV(XGB_0, XGB_param_0, verbose = False, 
                          scoring = 'neg_log_loss', cv = CV) # n_jobs = 6

XGB_tuned_0 = XGB_grid_0.fit(x_train, y_train, early_stopping_rounds = 15, 
                              eval_set = [[x_test, y_test]], 
                              eval_metric = 'logloss', verbose = False)